# Magma Keras to TensorRT

### Importing

In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential,load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile


import time 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import os
from os.path import isfile, exists, isdir, join

import shutil

Using TensorFlow backend.


### For Jetson TX2

In [2]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

tf.keras.backend.set_learning_phase(0)

In [3]:
CNN_MODEL = 'MagmaCnnClassifier.hdf5'
TARGET_SHAPE = 8
DATA_SHAPE = (100,100,3)
OPTIMIZER = Adam()
BATCH_SIZE = 256
EPOCH = 2

SAVED_MODEL_DIR = './saved_model/'
MODEL_NAME = CNN_MODEL

DATA_SHAPE = (100,100,3)
TRAIN_DIR = "./data/train"
TEST_DIR = "./data/test"
RESULT_PREDICTION_CALLBACK = None

# Data preprocessing
### create predict_dir by move random pictures from test_dir  
#### remove and copy test_dir

In [4]:
TEST_COPY_DIR = './data/test_dir'
PREDICT_DIR = './data/predict_dir'

In [ ]:
# delete folder if exist
if exists(TEST_COPY_DIR) and isdir(TEST_COPY_DIR):
    shutil.rmtree(TEST_COPY_DIR)
print('remove if exist test_dir success')

if exists(PREDICT_DIR) and isdir(PREDICT_DIR):
    shutil.rmtree(PREDICT_DIR)
print('remove if exist predict_dir success')

#copy test as test_dir, there are result as list of copy files
# from distutils.dir_util import copy_tree
# copy_tree('./data/test','./data/test_dir')
from subprocess import call
call(['cp','-a', TEST_DIR, TEST_COPY_DIR])
print('copy test to test_dir success')

#### create predict_dir and random moving images

In [5]:
#random select images
CATEGORIES = ['0','1','2','3','4','5','6','7']
IMAGES_PER_FOLDER = 1

In [ ]:
import random
for category in CATEGORIES:
    
    path_ct = join(TEST_COPY_DIR,category)
    path_pd = join(PREDICT_DIR, category)
    
    if not exists(path_pd):
        os.makedirs(path_pd)
    
    image_list = os.listdir(path_ct)
    random.shuffle(image_list)
    
    for img in image_list[:IMAGES_PER_FOLDER]:
        path_src = join(path_ct,img)
        path_des = join(path_pd,img)
        shutil.move(path_src, path_des)
    
    print('copy - category:',category, image_list[:IMAGES_PER_FOLDER])
    
    

# WorkFlow

### Keras to TensorRT
![alt text](pictures/Keras_to_TensorRT.png)

### Tensorflow to TensorRT
![alt text](pictures/tf-trt_workflow.png)

## a0. Convert Keras to Tensorflow model and a) Read input Tensorflow model
##### don't need to read input Tensorflow because i don't save .mega tensorflow file from keras
### Build Keras model (Declare + Train)
#### a.1) Declare Model

In [ ]:
model = Sequential([
    # Layer 1
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=DATA_SHAPE, name='input_tensor'),
    MaxPooling2D(pool_size=(2, 2)),
    # Layer 2
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    # Layer 3
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    # Layer 4
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    # flattening the model for fully connected layer
    Flatten(),
    Dropout(rate=0.5),
    # fully connected layer
    Dense(units=512, activation='relu'),
    # output layer
    Dense(units=TARGET_SHAPE, activation='softmax', name='output_tensor'),
])

# Compilile the network
model.compile(loss='categorical_crossentropy',
                    optimizer=OPTIMIZER,
                    metrics=['categorical_accuracy'])

model.summary()

#### a.2) create generator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1 / 255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1 / 255)

train_generator = train_datagen.flow_from_directory(
    directory = TRAIN_DIR ,
    target_size = DATA_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    directory = TEST_COPY_DIR, # use test data that be splited for prediction
    target_size = DATA_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

#### a.3) Train model

In [ ]:
sub_dir = time.strftime("%Y_%m_%d-%H_%M_%S", time.localtime())
tb_cb = TensorBoard(log_dir='./logs/'+sub_dir,
                    #histogram_freq=EPOCHS,
                    batch_size=BATCH_SIZE, 
                    write_grads=True,
                    write_images=True,
                    #update_freq='batch',
                    )
callbacks = [tb_cb]

# train_generator, val_generator declare Data preprocessing

model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//BATCH_SIZE,
                                epochs=EPOCH,
                                validation_data=val_generator,
                              validation_steps=val_generator.n//BATCH_SIZE,
                                callbacks=callbacks)

In [ ]:
# name of all nodes
[n.name for n in tf.get_default_graph().as_graph_def().node]

### --save model .hdf5

In [ ]:
model.save(SAVED_MODEL_DIR+CNN_MODEL)

### --load model .hdf5

In [ ]:
if isfile(SAVED_MODEL_DIR + MODEL_NAME):
    model = load_model(filepath= SAVED_MODEL_DIR + MODEL_NAME)
    model.summary()
else:
    raise Exception("--MODEL COULD NOT LOADED")

### Keras to TensorRT
![alt text](pictures/Keras_to_TensorRT.png)
## b) Convert to Frozen model .pb
#### b.1) declare function

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.
    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

#### b.2) Save the model to Protocol Buffers Format (.pb) as tf pb file

In [ ]:
for inp in model.inputs:
    print(inp.op.name)

In [ ]:
for out in model.outputs:
    print(out.op.name)

In [ ]:
from keras import backend as K
from tensorflow.python.platform import gfile

OUTPUTS = [out.op.name for out in model.outputs]
print(OUTPUTS)
print(OUTPUTS[0])
INPUTS = [inp.op.name for inp in model.inputs]
print(INPUTS)
print(INPUTS[0])

# model is used here 
# K.set_session(tf.Session(graph=model.output.graph)) 
init = K.tf.global_variables_initializer() 
K.get_session().run(init)

frozen_graph = freeze_session(K.get_session(), output_names=OUTPUTS)

In [ ]:
time_now = time.strftime("%d-%m-%Y_%H-%M-%S", time.localtime())
filename = "Magma_frozen_model_"+time_now+".pb"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("./saved_model/"+filename, 'wb') as f:
    f.write(frozen_graph.SerializeToString())
print("Frozen model is successfully stored!")

### Keras to TensorRT
![alt text](pictures/Keras_to_TensorRT.png)
## c) Optimize the frozen model to TensorRT graph

In [ ]:
# outputs = ["dense_2/Softmax"]
# outputs = ["output_tensor/Softmax"]

# convert (optimize) frozen model to TensorRT model
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,# frozen model
    outputs=OUTPUTS,
    # outputs=outputs,
    max_batch_size=2,# specify your max batch size
    max_workspace_size_bytes=2*(10**9),# specify the max workspace
    precision_mode="FP32") # precision, can be "FP32" (32 floating point precision) or "FP16"

In [ ]:
time_now = time.strftime("%d-%m-%Y_%H-%M-%S", time.localtime())
tr_filename = "TensorRT_Magma_model_"+time_now+".pb"

#write the TensorRT model to be used later for inference
with gfile.FastGFile("./saved_model/"+tr_filename, 'wb') as f:
    f.write(trt_graph.SerializeToString())
print("TensorRT model is successfully stored!")

### Count how many nodes/operations before and after optimization

In [ ]:
# check how many ops of the original frozen model
all_nodes = len([1 for n in frozen_graph.node])
print("numb. of all_nodes in frozen graph:", all_nodes)

# check how many ops that is converted to TensorRT engine
trt_engine_nodes = len([1 for n in trt_graph.node if str(n.op) == 'TRTEngineOp'])
print("numb. of trt_engine_nodes in TensorRT graph:", trt_engine_nodes)
all_nodes = len([1 for n in trt_graph.node])
print("numb. of all_nodes in TensorRT graph:", all_nodes)

# Visualize the original and optimized graphs
Using [netron](https://lutzroeder.github.io/netron/), the web application for vitsulaize model graph by upload .pb file.